**Mapper and Reducer with MapReduce**

The Python file mimics the process of MapReduce task, specifically writing map and reduce functions (without distributing to several machines) to mimic the process of mapper and reducer. The task is to count the number of occurrences of each word in a text file.

**Reading the file**

The input here is a text document (around 13,000 lines) which includes several paragraphs. It is a raw data which requires some data cleaning works to prepare it for next step.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
inputfile = open('/content/drive/My Drive/MIS/Fall 2020/IDS 561 Big data/Pride_and_Prejudice.txt',"r")
text = inputfile.read()

**Data Cleaning Function**

This step includes data cleaning tasks such as removing numbers from text, punctuations and special symbols, uppercase to lower case.

In [3]:
def data_clean(text):
  RemoveNumbers = ''.join([i for i in text if not i.isdigit()]) # Removing the numbers from text
  RemoveNumbers = text.lower()                                  # Changing case of text to lower case
  import re
  onlyText = re.sub(r"[^a-z\s]+",' ',RemoveNumbers)             # Removing punctuation marks from text
  finaltext = "".join([s for s in onlyText.strip().splitlines(True) if s.strip()]) # Removing all null lines
  return finaltext

**Data Spilt Function**

Output of data cleaning function fed into the data split function with two parts to output two separated subsets: Part1 (first 5000 lines of 13000) and Part2 (5001 and beyond lines)

In [4]:
def splitlines(text,a):
  linessplit = text.splitlines()   # Splitting the lines into a list
  part1 = linessplit[0:5000]       # Part1 includes the first 5000 lines of the text file
  part2 = linessplit[50001:]       # Part2 includes the rest of the text: lines 5001 and beyond
  return part1,part2

**Mapper Function**

The mapper function produces a set of key-value pairs for Part1 and Part2 subsets respectively.

In [5]:
def mapper(text,out_queue):
  keyvalue = []
  for i in text:
    wordssplit = i.split()
    for j in wordssplit:
      keyvalue.append([j,1])      # Appending every word in the line with 1 and storing it in ["word", 1] format in a nested list
  out_queue.put(keyvalue)

**Sort Function**

Sort by key of Part1 and Part2 together, with an ascending sort order

In [6]:
def sortedlists(list1,list2):
  out = list1 + list2             # Combining the two input lists into a single list
  out.sort(key= lambda x :x[0])   # Sorting the lists based on the first element of the list "word"
  return out

**Partition Function**

The output of sort function is fed into the partition function to give two ascending ordered partitions

In [7]:
def partition(sorted_list) :
 sort1out = []
 sort2out = []
 for i in sorted_list:
    if i[0][0] < 'n':            # Partitioning the sorted word list into two separate lists 
      sort1out.append(i)         # All the words starting with letter “a” to “m” are sent to Reducer1, and the others (“n” to “z”) are sent to Reducer2.
    else : sort2out.append(i)
 return sort1out,sort2out

**Reducer Function**

Collect all values belonging to the key and count the frequency of words for the two ordered partitions. The function output is word frequency of the ordered partitions

In [8]:
def reducer(part_out1,out_queue) :
  sum_reduced = []
  count = 1
  for i in range(0,len(part_out1)):
    if i < len(part_out1)-1:
      if part_out1[i] == part_out1[i+1]:
       count = count+1                              #Counting the number of words
      else : 
       sum_reduced.append([part_out1[i][0],count])  #Appending the word along with count to sum_reduced list as ["word",count]
       count = 1 
    else: sum_reduced.append(part_out1[i])          #Appending the last word to the output list    
  out_queue.put(sum_reduced)

**Using multi-thread function**

Multithreading is defined as the ability of a processor to execute multiple threads concurrently. It takes two user inputs as arguments. 

In [9]:
import threading
import queue
def multi_thread_function(func,map1_input,map2_input):  #func is the function to be used with two threads taking two inputs map1_input and map2_input
  my_queue1 = queue.Queue()  # Using a queue to store the values of mapper output and use them in sort function
  my_queue2 = queue.Queue()
  t1 = threading.Thread(target=func, args=(map1_input,my_queue1)) 
  t2 = threading.Thread(target=func, args=(map2_input,my_queue2))  
  t1.start()                 # Executing input1
  t2.start()                 # Executing input2 to run simultaneously with input1
  t1.join()                  # Waiting for input1 to be executed
  t2.join()                  # Waiting for input2 to be executed
  list1out = my_queue1.get() # Getting values from queue into a variable to return its value
  list2out = my_queue2.get()
  return list1out,list2out

**Main Function**

Final result of word counting by wrapping all the steps together and combining the output of the two partitions together

In [10]:
def main_function(text):  
  cleantext = data_clean(text)
  linessplit = splitlines(cleantext,5000)
  mapperout = multi_thread_function(mapper,linessplit[0],linessplit[1]) 
  sortedwords = sortedlists(mapperout[0],mapperout[1])
  slicedwords = partition(sortedwords)
  reducerout = multi_thread_function(reducer,slicedwords[0],slicedwords[1])
  return reducerout[0]+reducerout[1]

output = main_function(text)
import pandas as pd
pd.DataFrame(output).to_csv("Output.csv",index=False,header = ["Word","Frequency"]) # Saving file as a .csv file in the current directory